# Dockerizing ML Models

In this notebook, you'll learn how to **containerize Machine Learning models** using **Docker** — making them portable, reproducible, and ready for deployment.

## 🎯 Objectives
- Understand what Docker is and why it’s useful for ML workflows.
- Learn how to create a Dockerfile for your model.
- Build and run a Docker container serving your ML API.
- Manage dependencies and environment consistency across systems.

---

## ⚙️ 1. What is Docker?

Docker allows you to **package applications and dependencies into containers**, which run the same way on any system.

**Benefits for ML:**
- Ensures consistent environment (no more *“it works on my machine”* issues)
- Portable deployment on servers or cloud
- Lightweight and faster than full virtual machines

🧩 Example use case: Packaging a FastAPI model server into a Docker container.

## 🧠 2. Prepare the Model and API

We’ll reuse the Iris classifier and FastAPI app created earlier.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import joblib

# Train and save the model
iris = load_iris()
model = RandomForestClassifier()
model.fit(iris.data, iris.target)
joblib.dump(model, 'iris_model.pkl')
print('✅ Model saved as iris_model.pkl')

Now let’s create our **FastAPI app** (`app.py`) that will serve predictions.

In [ ]:
%%writefile app.py
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np

model = joblib.load('iris_model.pkl')
app = FastAPI(title='Iris Classifier API')

class IrisInput(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float

@app.get('/')
def root():
    return {'message': 'Welcome to the Iris Model API'}

@app.post('/predict')
def predict(data: IrisInput):
    X = np.array([[data.sepal_length, data.sepal_width, data.petal_length, data.petal_width]])
    pred = model.predict(X)
    return {'prediction': int(pred[0])}

## 🐋 3. Writing the Dockerfile

A **Dockerfile** defines how to build your container image. Let’s create one that:
- Uses a base Python image
- Installs dependencies
- Copies code and model
- Runs the FastAPI app via Uvicorn

In [ ]:
%%writefile Dockerfile
FROM python:3.10
WORKDIR /app

# Copy files
COPY app.py /app/
COPY iris_model.pkl /app/

# Install dependencies
RUN pip install fastapi uvicorn scikit-learn joblib

# Expose port
EXPOSE 8000

# Run API
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]

## 🔨 4. Building the Docker Image

Run the following command in your terminal (in the same folder as `Dockerfile`):

```bash
docker build -t iris-ml-api .
```

This will:
1. Download the Python base image.
2. Copy code & model.
3. Install dependencies.
4. Package everything into a Docker image named **iris-ml-api**.

## ▶️ 5. Running the Container

Once built, start the container using:
```bash
docker run -d -p 8000:8000 iris-ml-api
```

Visit:
- Swagger UI → http://127.0.0.1:8000/docs
- Root endpoint → http://127.0.0.1:8000/

🎉 You now have your ML model running inside a **Docker container!**

## 🧪 6. Testing the API Inside Docker

You can send requests using Python or `curl`.

In [ ]:
import requests

url = 'http://127.0.0.1:8000/predict'
sample = {
    'sepal_length': 5.1,
    'sepal_width': 3.5,
    'petal_length': 1.4,
    'petal_width': 0.2
}

response = requests.post(url, json=sample)
print(response.json())

## ☁️ 7. Deploying Dockerized Model to Cloud

**Options:**
- **Docker Hub** → Push your image for reuse:
  ```bash
  docker tag iris-ml-api username/iris-ml-api:v1
  docker push username/iris-ml-api:v1
  ```

- **Cloud Platforms:**
  - AWS Elastic Beanstalk
  - Google Cloud Run
  - Azure Container Apps
  - Render / Railway (supports Dockerfile-based deployments)

**Kubernetes (Advanced):**
- Once Dockerized, you can deploy the container as a Pod in a K8s cluster using a `Deployment` and `Service` YAML manifest.

## 🔒 8. Best Practices
- Use **`.dockerignore`** to avoid copying unnecessary files.
- Pin dependency versions for reproducibility.
- Use multi-stage builds to reduce image size.
- Store model weights in a versioned artifact store (e.g., DVC, MLflow).
- Add health checks in production containers.

---
✅ **You have successfully Dockerized your ML Model!**

Next → **07-CI_CD_with_GitHub_Actions.ipynb** 🚦